In [ ]:
import os
import netCDF4 as nc
import numpy as np
import rasterio
from rasterio.transform import from_origin
from datetime import datetime

fldrs = ['03_07','07_11','11_15','15_19','19_21']
names = [2003, 2004, 2005, 2006, 2007, 2007, 2008, 2009, 2010, 2011, 2011, 2012, 2013, 2014, 2015, 2015, 2016, 2017, 2018, 2019, 2019, 2020, 2021]
skips = [5, 10, 15, 20]
# variable = 'Rainfall_ET_M'
# print(names)
netcdfs = ['AFR_ETa_SSEBop_yearly', 'AFR_P_CHIRPS_yearly', 'sro_yearly' ]

for netcdf in netcdfs:
        i = -1

        for fldr in fldrs:


            # Replace with your input NetCDF file path
            nc_file_path = "/efs/CWA/netcdf_files_%s_/%s.nc"%(fldr, netcdf)

            # Open the NetCDF file for reading
            nc_dataset = nc.Dataset(nc_file_path, 'r')

            variable = list(nc_dataset.variables.keys())[-1]

            # Replace with the output directory where you want to save the GeoTIFF files
            output_dir = "/home/iwmi-wa/tmp_1tb/TIFF/%s/"%(variable)

            # Create the output directory if it doesn't exist
            os.makedirs(output_dir, exist_ok=True)

            # Loop through each band and convert it to a GeoTIFF
            for band_index in range(len(nc_dataset.variables['time'])):

                i = i + 1
                if i in skips: continue

                band_data = nc_dataset.variables[variable][band_index]  # Using the 'Rainfall_ET_M' variable
                year = int(nc_dataset.variables['time'][band_index])  # Extracting the year from the 'time' variable

                output_tiff_name = os.path.join(output_dir, f"{variable}_{names[i]}.01.01.tif")

                # Define a transform for EPSG:4326
                transform = from_origin(
                    nc_dataset.variables['longitude'][0],
                    nc_dataset.variables['latitude'][0],
                    nc_dataset.variables['longitude'][1] - nc_dataset.variables['longitude'][0],
                    - nc_dataset.variables['latitude'][1] + nc_dataset.variables['latitude'][0]
                )


                # Specify compression and other options
                profile = {
                    'driver': 'GTiff',
                    'height': band_data.shape[0],
                    'width': band_data.shape[1],
                    'count': 1,
                    'dtype': str(band_data.dtype),
                    'crs': 'EPSG:4326',  # Common CRS for WGS 84
                    'transform': transform,
                    'compress': 'LZW',  # Change to your desired compression method (e.g., 'LZW', 'DEFLATE', 'JPEG', etc.)
                }

                # Write the band to a GeoTIFF file with compression
                with rasterio.open(output_tiff_name, 'w', **profile) as dst:
                    dst.write(band_data, 1)

            # Close the NetCDF file
            nc_dataset.close()


In [ ]:
print(list(nc_dataset.variables.keys())[-1])

In [ ]:
# List available variable names and their attributes in the NetCDF file
for var_name, var in nc_dataset.variables.items():
    print(f"Variable: {var_name}")
    for attr_name in var.ncattrs():
        print(f"  Attribute: {attr_name} = {var.getncattr(attr_name)}")


In [ ]:
[*range(1,13)]

In [2]:
import os
import netCDF4 as nc
import numpy as np
import rasterio
from rasterio.transform import from_origin
from datetime import datetime

fldrs = ['03_07','07_11','11_15','15_19','19_21']
names = [ 2003, 2004, 2005, 2006, 2007, 2007, 2008, 2009, 2010, 2011, 2011, 2012, 2013, 2014, 2015, 2015, 2016, 2017, 2018, 2019, 2019, 2020, 2021]
skips = [5, 10, 15, 20]
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
# variable = 'Rainfall_ET_M'
# print(names)
netcdfs = ['i_monthly','evaporation']
# netcdfs = ['AFR_ETa_SSEBop', 'AFR_P_CHIRPS']

for netcdf in netcdfs:
    
        i = 0

        for fldr in fldrs:


            # Replace with your input NetCDF file path
            nc_file_path = f"/efs/CWA/ncdf_files_sheet2_%s/%s.nc"%(fldr, netcdf)

            # Open the NetCDF file for reading
            nc_dataset = nc.Dataset(nc_file_path, 'r')

            variable = list(nc_dataset.variables.keys())[-1]

            # Replace with the output directory where you want to save the GeoTIFF files
            output_dir = f"/efs/Data_extraction/For_ET_sheets/Tifs/%s/"%(variable)

            # Create the output directory if it doesn't exist
            os.makedirs(output_dir, exist_ok=True)
            
            j = -1

            # Loop through each band and convert it to a GeoTIFF
            for band_index in range(len(nc_dataset.variables['time'])):
                
                j = j + 1

                if (j<12 and i>0): continue
                
                m = ((j+1)%12)-1
                
                y = int(j/12) + i*5
                
#                 print(band_index,m)

                band_data = nc_dataset.variables[variable][band_index]  # Using the 'Rainfall_ET_M' variable
#                 year = int(nc_dataset.variables['time'][band_index])  # Extracting the year from the 'time' variable

                output_tiff_name = os.path.join(output_dir, f"{variable}_{names[y]}.{months[m]}.01.tif")
                print(band_index,m,y,output_tiff_name)

                # Define a transform for EPSG:4326
                transform = from_origin(
                    nc_dataset.variables['longitude'][0],
                    nc_dataset.variables['latitude'][0],
                    nc_dataset.variables['longitude'][1] - nc_dataset.variables['longitude'][0],
                    - nc_dataset.variables['latitude'][1] + nc_dataset.variables['latitude'][0]
                )


                # Specify compression and other options
                profile = {
                    'driver': 'GTiff',
                    'height': band_data.shape[0],
                    'width': band_data.shape[1],
                    'count': 1,
                    'dtype': str(band_data.dtype),
                    'crs': 'EPSG:4326',  # Common CRS for WGS 84
                    'transform': transform,
                    'compress': 'LZW',  # Change to your desired compression method (e.g., 'LZW', 'DEFLATE', 'JPEG', etc.)
                }

                # Write the band to a GeoTIFF file with compression
                with rasterio.open(output_tiff_name, 'w', **profile) as dst:
                    dst.write(band_data, 1)

#             Close the NetCDF file
            nc_dataset.close()
            
            i =i+1


0 0 0 /efs/Data_extraction/For_ET_sheets/Tifs/I/I_2003.01.01.tif
1 1 0 /efs/Data_extraction/For_ET_sheets/Tifs/I/I_2003.02.01.tif
2 2 0 /efs/Data_extraction/For_ET_sheets/Tifs/I/I_2003.03.01.tif
3 3 0 /efs/Data_extraction/For_ET_sheets/Tifs/I/I_2003.04.01.tif
4 4 0 /efs/Data_extraction/For_ET_sheets/Tifs/I/I_2003.05.01.tif
5 5 0 /efs/Data_extraction/For_ET_sheets/Tifs/I/I_2003.06.01.tif
6 6 0 /efs/Data_extraction/For_ET_sheets/Tifs/I/I_2003.07.01.tif
7 7 0 /efs/Data_extraction/For_ET_sheets/Tifs/I/I_2003.08.01.tif
8 8 0 /efs/Data_extraction/For_ET_sheets/Tifs/I/I_2003.09.01.tif
9 9 0 /efs/Data_extraction/For_ET_sheets/Tifs/I/I_2003.10.01.tif
10 10 0 /efs/Data_extraction/For_ET_sheets/Tifs/I/I_2003.11.01.tif
11 -1 0 /efs/Data_extraction/For_ET_sheets/Tifs/I/I_2003.12.01.tif
12 0 1 /efs/Data_extraction/For_ET_sheets/Tifs/I/I_2004.01.01.tif
13 1 1 /efs/Data_extraction/For_ET_sheets/Tifs/I/I_2004.02.01.tif
14 2 1 /efs/Data_extraction/For_ET_sheets/Tifs/I/I_2004.03.01.tif
15 3 1 /efs/Data_e

FileNotFoundError: [Errno 2] No such file or directory: b'/efs/CWA/ncdf_files_sheet2_07_11/i_monthly.nc'

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jan 24 11:01:05 2020

@author: cmi001
"""

# %% netCDF files from
import os
import glob
import sys

sys.path.append(r'/efs/CWA/scripts_aridity')
import  createNC_cmi
import time

Tstart = time.time()

basedir = r"/home/iwmi-wa/tmp_1tb/Monthly_tifs_"

pathP = os.path.join(basedir,'P')
pathETa = os.path.join(basedir,'ETa' )
pathETinr = os.path.join(basedir,'Incremental_ET_M' )
pathETtrain = os.path.join(basedir,'Rainfall_ET_M' )
pathF = os.path.join(basedir,'fraction' )
pathLU = os.path.join(basedir,'LU')
pathT = os.path.join(basedir,'transpiration' )
pathE = os.path.join(basedir,'evaporation' )
pathSRO = os.path.join(basedir,'SRO_M')

template = r"/efs/CWA/static_datasets/basin_mask/AFR_basinmask_1km.tif"

shp_path = r"/efs/CWA/static_datasets/shapefile/Africa_adm0_buffer.shp"
save_location = r"/home/iwmi-wa/tmp_1tb/Monthly_nc_/"


name = 'AFR'

datasets = {
#                     'P'      :   [pathP, 
#                                  ('time','latitude', 'longitude'), 
#                                        {'units': 'mm/month', 'source': 'CHIRPS_yearly', 'quantity':'P'}],
                    'ET'     :   [pathETa, 
                                   ('time','latitude', 'longitude'), 
                                   {'units': 'mm/month', 'source': 'SSEBop_yearly', 'quantity':'ETa'}],   
#                     'LU'     :   [pathLU,
#                                    ('time','latitude', 'longitude'), 
#                                    {'units': 'None', 'source': 'WA', 'quantity':'LU'}], 
    
#                     'ETincr'     :   [pathETinr,
#                                    ('time','latitude', 'longitude'), 
#                                    {'units': 'None', 'source': 'ETincr_yearly', 'quantity':'Incremental_ET_M'}],
#                     'ETrain'     :   [pathETtrain,
#                                    ('time','latitude', 'longitude'), 
#                                    {'units': 'None', 'source': 'ETrain_yearly', 'quantity':'Rainfall_ET_M'}],
#                     'F'     :   [pathF,
#                                     ('time','latitude', 'longitude'), 
#                                    {'units': 'None', 'source': 'F', 'quantity':'fraction'}],
                    'E'     :   [pathE,
                                   ('time','latitude', 'longitude'), 
                                   {'units': 'None', 'source': 'E', 'quantity':'evaporation'}],
                    'T'     :   [pathT,
                                   ('time','latitude', 'longitude'), 
                                   {'units': 'None', 'source': 'T', 'quantity':'evaporation'}]
#                     'SRO'     :   [pathSRO,
#                                    ('time','latitude', 'longitude'), 
#                                    {'units': 'None', 'source': 'SRO_yearly', 'quantity':'SRO_M'}]
          }

nc_files = []

if not os.path.exists(save_location):
    os.makedirs(save_location)

for d in datasets:
    start = time.time()
    filesAll = glob.glob(os.path.join(datasets[d][0],'*.tif'))
    dataset = {d: datasets[d]}
    fname = name+'_'+datasets[d][2]['quantity']+'_'+ datasets[d][2]['source']+'.nc'
    nc_file = os.path.join(save_location, fname)
    nc_files.append(nc_file)
    succes = createNC_cmi.make_netcdf(nc_file, dataset, shp_path, template, name)    
    end = time.time()
    print('Time for {} - {}'.format(d, end - start))
    
Tend = time.time()
print('Total time for execution - ',Tend - Tstart)